In [15]:
import torch
import torch.nn as nn
import math
import numpy as np
from collections import Counter

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class TransformerTextGenerator(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, max_len=100):
        super(TransformerTextGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, vocab_size)
        self.d_model = d_model
        self.max_len = max_len

    def forward(self, src, src_mask=None):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.fc(output)
        return output

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def prepare_data(text, seq_length=30):
    words = text.split()
    word_counts = Counter(words)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    word_to_idx = {word: i for i, word in enumerate(vocab)}
    idx_to_word = {i: word for i, word in enumerate(vocab)}

    sequences = []
    for i in range(seq_length, len(words)):
        seq = words[i-seq_length:i+1]
        sequences.append([word_to_idx[word] for word in seq])

    return np.array(sequences), word_to_idx, idx_to_word, len(vocab)

def train_model_transformer(model, sequences, vocab_size, seq_length=30, epochs=100, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

    model.train()

    for epoch in range(epochs):
        total_loss = 0
        batch_count = 0

        for sequence in sequences:
            optimizer.zero_grad()

            # Подготовка входных данных и целевых значений
            input_seq = torch.tensor(sequence[:-1]).unsqueeze(0)  # [1, seq_len-1]
            target = torch.tensor(sequence[1:])  # [seq_len-1]

            # Создаем маску для автогрессии
            src_mask = generate_square_subsequent_mask(input_seq.size(1))

            # Forward pass
            output = model(input_seq, src_mask)  # [1, seq_len-1, vocab_size]

            # Изменяем размерности для вычисления loss
            output = output.view(-1, vocab_size)  # [(seq_len-1), vocab_size]
            loss = criterion(output, target)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            batch_count += 1

        scheduler.step()
        avg_loss = total_loss / batch_count

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {avg_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

            # Тест генерации
            if avg_loss < 6.0:  # Когда модель немного обучилась
                test_text = generate_text_transformer(model, "машинное обучение", word_to_idx, idx_to_word, seq_length, 10)
                print(f"Тест генерации: {test_text}")

def generate_text_transformer(model, start_text, word_to_idx, idx_to_word, seq_length, num_words=20, temperature=0.8):
    model.eval()
    words = start_text.split()

    if len(words) == 0:
        return ""

    # Оставляем только слова из словаря
    words = [word for word in words if word in word_to_idx]
    if len(words) == 0:
        return ""

    # Если слов меньше seq_length, дополняем padding'ом
    if len(words) < seq_length:
        padded_words = ['<PAD>'] * (seq_length - len(words)) + words
    else:
        padded_words = words[-seq_length:]

    generated_text = words.copy()

    with torch.no_grad():
        current_sequence = [word_to_idx[word] for word in padded_words]

        for _ in range(num_words):
            input_seq = torch.tensor(current_sequence).unsqueeze(0)  # [1, seq_length]

            # Создаем маску
            src_mask = generate_square_subsequent_mask(input_seq.size(1))

            # Получаем предсказания
            output = model(input_seq, src_mask)  # [1, seq_length, vocab_size]

            # Берем предсказание для последнего слова в последовательности
            next_word_logits = output[0, -1, :]  # [vocab_size]

            # Применяем temperature
            next_word_probs = torch.softmax(next_word_logits / temperature, dim=0)

            # Выбираем следующее слово
            next_word_idx = torch.multinomial(next_word_probs, 1).item()
            next_word = idx_to_word.get(next_word_idx, "<UNK>")

            # Обновляем последовательность
            generated_text.append(next_word)
            current_sequence = current_sequence[1:] + [next_word_idx]

    return ' '.join(generated_text)

# Пример использования
text = """машинное обучение это интересная область искусственного интеллекта
которая позволяет компьютерам обучаться на данных и делать прогнозы
нейронные сети являются мощным инструментом для решения сложных задач
глубокое обучение использует многослойные нейронные сети
для распознавания образов и обработки естественного языка"""

print("Подготовка данных...")
sequences, word_to_idx, idx_to_word, vocab_size = prepare_data(text, seq_length=20)
print(f"Размер словаря: {vocab_size}")
print(f"Количество последовательностей: {len(sequences)}")

# Добавляем токен паддинга в словарь
word_to_idx['<PAD>'] = len(word_to_idx)
idx_to_word[len(idx_to_word)] = '<PAD>'
vocab_size += 1

# Создаем модель Transformer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerTextGenerator(
    vocab_size=vocab_size,
    d_model=128,
    nhead=8,
    num_layers=3,
    max_len=50
).to(device)

print(f"Модель создана на устройстве: {device}")
print(f"Количество параметров: {sum(p.numel() for p in model.parameters()):,}")

print("Начало обучения Transformer...")
train_model_transformer(model, sequences, vocab_size, seq_length=20, epochs=100, lr=0.001)

# Финальная генерация
print("\nФинальная генерация:")
result = generate_text_transformer(model, "машинное обучение", word_to_idx, idx_to_word, 20, 25)
print(result)

Подготовка данных...
Размер словаря: 33
Количество последовательностей: 18
Модель создана на устройстве: cpu
Количество параметров: 1,787,810
Начало обучения Transformer...
Epoch 0, Loss: 0.9339, LR: 0.001000
Тест генерации: машинное обучение использует многослойные нейронные сети являются мощным инструментом для естественного и
Epoch 10, Loss: 0.0024, LR: 0.001000
Тест генерации: машинное обучение использует многослойные нейронные сети являются мощным инструментом для решения сложных
Epoch 20, Loss: 0.0011, LR: 0.001000
Тест генерации: машинное обучение использует многослойные нейронные сети являются мощным инструментом для решения сложных
Epoch 30, Loss: 0.0007, LR: 0.000100
Тест генерации: машинное обучение использует многослойные нейронные сети являются мощным инструментом для решения сложных
Epoch 40, Loss: 0.0006, LR: 0.000100
Тест генерации: машинное обучение использует многослойные нейронные сети являются мощным инструментом для решения сложных
Epoch 50, Loss: 0.0006, LR: 0.000